* our team crawled data from sites Dabang(this is code to run on the ubuntu enviroment)

 * Save every data in  MongoDB
 * 1st saved all the urls of the site where we can find the information of individual room near the universities in Seoul python pacakge Selenium
 * By using Selenium package we inserted universities names in the search box
![GitHub Logo](image1.png) 
 * Also we can turn to the next page
![GitHub Logo](image2.png) 

In [ ]:
from selenium.webdriver.support.ui import Select
from  pyvirtualdisplay import Display
import time
from pymongo import MongoClient
from selenium import webdriver
from collections import Counter


class Href_Scretcher_Dabang(object):
    
    def __init__(self, university_lst):
        self.university_lst = university_lst
    
       
    def extract_href(self):
        href_lst = []
        display = Display(visible=0, size=(800,600))
        display.start()
        driver = webdriver.Chrome()
        driver.get('https://www.dabangapp.com/search#/map')
        for i in self.university_lst:
            # Insert university name in the search_box
            try:
                text_box = driver.find_element_by_xpath("//input[@class='SearchForm-input form-control']")
                text_box.clear()
                address =  i
                address = address.decode('utf8')
                text_box.send_keys(address)
                time.sleep(5)
            except:
                continue
            # Click the box if the name we insert matched    
            try:
                click_box = driver.find_element_by_xpath('''//ul[@class='SearchForm-list search-items']/li/span[@class='SearchForm-item name full']''')
                text = click_box.text
            except:
                print 'text_error' 
            if address in text:    
                click_box.click() 
            else:
                print 'not match'
                continue    
            # Wait untill page loaded successfully
            time.sleep(3)
            try:
                region_box = driver.find_elements_by_class_name("Room-item")
                # added all the urls  
                for i,room_info in enumerate(region_box):
                    try:
                        link = room_info.find_element_by_tag_name('a')
                        href = link.get_attribute('href')
                        href_lst.append(href)
                    except:
                        print 'no a tag with href'                
                        
                while True:
                    try:
                        time.sleep(3)
                        # find the next icon to turn to the next pages
                        next_icon = driver.find_element_by_xpath("//ul[@class='Pagination']//a[@class='Pagination-item Pagination-item--next']")
                        next_icon.click()
                        region_box = driver.find_elements_by_class_name("Room-item")
                        for i,room_info in enumerate(region_box):
                            try:
                                link = room_info.find_element_by_tag_name('a')
                                href = link.get_attribute('href')
                                href_lst.append(href)
                            except:
                                print 'no a tag with href'
                                continue
                    #If there is no active next icon break from the loop    
                    except: 
                        print 'no next icon'
                        break

                    
            
            except:
                print 'Nothing to do'
        driver.quit()
        return href_lst
    
    def extract_final_lst(self):
        href_lst = self.extract_href()
        href_lst = set(href_lst)
        self.final_lst = list(href_lst)
        return final_lst
    
    def href_to_mongo(self):
        mongo = MongoClient('Your Mongo DB Repository', 27017)
        database = mongo.Hrefs.urls
        href_data = []
        st = set()
        # insert the new url that is not found in the Mongo DB
        for i in database.find():
            st.add(i['url'])
        for i in self.final_lst:
            url = {}
            if i in st:
                continue
            url['url'] = i
            href_data.append(url)
        database.insert_many(href_data)
        
    # Function to write the collected data to the textfile     
    def write_file(self, file_name):
        with open(file_name,'w') as txtfile:
            for i in self.final_lst:
                url = i + ' '
                txtfile.write(url)
print 'start'
u_lst = ['서울교육대학교', '서울대학교', '서울과학기술대학교','육군사관학교','한국방송통신대학교',
 '한국예술종합학교','한국체육대학교','KAIST','서울시립대학교','가톨릭대학교','감리교신학대학교',
 '건국대학교','경기대학교','경희대학교','고려대학교','광운대학교','국민대학교','그리스도대학교',
 '덕성여자대학교','동국대학교','동덕여자대학교','명지대학교','삼육대학교','상명대학교',
 '서강대학교','서경대학교','서울기독대학교','서울여자대학교','성공회대학교','성균관대학교',
 '성신여자대학교','세종대학교','숙명여자대학교','숭실대학교','연세대학교','이화여자대학교',
 '장로회신학대학교','중앙대학교','총신대학교','추계예술대학교','한국성서대학교',
 '한국외국어대학교','한성대학교','한양대학교','한영신학대학교','홍익대학교']
dabang.extract_final_lst()
dabang.href_to_mongo()
print 'Sucess'


* After we have stored all the urls in to the MongoDB we added all the datas found

In [ ]:

import numpy as np
import pandas as pd
from pymongo import MongoClient
from operator import itemgetter
from bs4 import BeautifulSoup
import requests
import json
import re
from pymongo import MongoClient


class Href_to_data(object):
    def __init__(self):
        pass
        
    def href_to_mongodb(self):
        mongo = MongoClient('Your Mongo DB Repository', 27017)
        hrefbase = mongo.Hrefs.urls
        house_info = mongo.dabang.room_db2
        ls = set() 
        try:
            for i in house_info.find():
                ls.add('https://www.dabangapp.com/room/'+ i['id'])
        except:
            ls = []
        
        house_data = [] 
        for  href in  hrefbase.find():
            if href['url'] in ls:
                continue
            try:
                web = requests.get(href['url'])
                x = re.search('dabang.web.detail\((.*}})',web.content) 
                jsonform = x.group(1)
            except:continue
            try:
                jsonf = json.loads(jsonform)
            except:continue
            housedict = {}
            housedict['id'] = jsonf['room']['id']
            housedict['address']= jsonf['room']['address']
            housedict['loc'] = (jsonf['room']['location'])
            housedict['room_size'] = jsonf['room']['room_size']
            housedict['room_floor'] = jsonf['room']['room_floor']
            housedict['building_floor'] = jsonf['room']['building_floor']
            housedict['room_options'] = jsonf['room']['room_options']
            housedict['room_type'] = jsonf['room']['room_type_str']
            try:
                housedict['metro_count'] = jsonf['room']['near'][0]['total']
                housedict['d_to_metro'] = min(jsonf['room']['near'][0]['pois'],key = itemgetter('distance'))['distance']
                housedict['d_to_conv'] = min(jsonf['room']['near'][1]['pois'],key = itemgetter('distance'))['distance']
                housedict['d_to_coffe'] = min(jsonf['room']['near'][2]['pois'],key = itemgetter('distance'))['distance']
                housedict['d_to_bank'] = min(jsonf['room']['near'][3]['pois'],key = itemgetter('distance'))['distance']
            except:
                print 'fail'
                housedict['metro_count'] = np.nan
                housedict['d_to_metro'] = np.nan
                housedict['d_to_conv'] = np.nan
                housedict['d_to_coffe'] = np.nan
                housedict['d_to_bank']= np.nan
	    
            try:
                housedict['elevator'],housedict['parking'],housedict['animal'] = jsonf['room']['elevator'],jsonf['room']['parking'],jsonf['room']['animal']
            except:
                housedict['elevator'],housedict['parking'] = np.nan, np.nan
            try:
                if jsonf['room']['maintenance']:
                    mc = jsonf['room']['maintenance_cost']
                else: 
                    mc = 0
                if mc is None: 
                    mc = 0
                deposit = jsonf['room']['price_info'][0][0] 
                if deposit is None: 
                    deposit = 0
                rent = jsonf['room']['price_info'][0][1] 
                if rent is None :
                    rent = 0
                price = deposit + (rent + mc) * 100
            except:continue
            housedict['price'] = price
            try:
                house_info.insert_one(housedict)
                print 'Success'
            except: 
                print 'fail'
                continue
print 'start'    
data = Href_to_data()
data.href_to_mongodb()